In [1]:
import pywt
import numpy as np
from matplotlib import pyplot as plt

In [2]:
import glob
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
%matplotlib inline

In [3]:
reading_dates=pd.date_range('2011-01-01',freq='W',periods=28)
print(reading_dates)

DatetimeIndex(['2011-01-02', '2011-01-09', '2011-01-16', '2011-01-23',
               '2011-01-30', '2011-02-06', '2011-02-13', '2011-02-20',
               '2011-02-27', '2011-03-06', '2011-03-13', '2011-03-20',
               '2011-03-27', '2011-04-03', '2011-04-10', '2011-04-17',
               '2011-04-24', '2011-05-01', '2011-05-08', '2011-05-15',
               '2011-05-22', '2011-05-29', '2011-06-05', '2011-06-12',
               '2011-06-19', '2011-06-26', '2011-07-03', '2011-07-10'],
              dtype='datetime64[ns]', freq='W-SUN')


In [4]:
files=glob.glob("csv/*.csv")
data=[]
names=[]
for file in files:
    df=read_csv(file,header=None,names=['Av','Mx_rt','Mn_rt','Avg_rt','th'])
    df['R_dates']=reading_dates
    df=df.set_index('R_dates')
    names.append(file)
    data.append(df)

In [10]:
#denoise data values
denoised_dfs=[]

for i in range(len(data)):
    denoised_df=[]
    for j in range(5):       ## number of attributes
        ts_Q_attr=data[i][data[i].columns[j]]
        ts_Q_attr=ts_Q_attr.values
        (ca, cd) = pywt.dwt(ts_Q_attr,'haar')
        cat=pywt.threshold(ca, np.std(ca)/2,mode='soft')
        cdt=pywt.threshold(cd, np.std(cd)/2,mode='soft')
        ts_Q_attr_rec = pywt.idwt(cat, cdt, 'haar')
        denoised_df.append(ts_Q_attr_rec.round(2))
    denoised_df=DataFrame(denoised_df)
    denoised_df=denoised_df.T
    denoised_df.to_csv('csv_denoised/'+names[i][4:])
    denoised_dfs.append(denoised_df)



/home/ubuntu/anaconda3/envs/tensorflow/lib/python3.6/site-packages/pywt/_thresholding.py:23: RuntimeWarning: invalid value encountered in true_divide
  thresholded = (1 - value/magnitude)


In [42]:
nan_contained_files=[]
for i in range(len(denoised_dfs)):
    for j in range(28):
        for k in range(5):
            if np.isnan(denoised_dfs[i].iloc[j][k]):
                nan_contained_files.append(names[i])
    
    

nan_contained_files=list(set( nan_contained_files))
nan_contained_files

['csv/service73.csv',
 'csv/service76.csv',
 'csv/service75.csv',
 'csv/service74.csv',
 'csv/service26.csv']

In [43]:
len(nan_contained_files)

5

In [45]:
import os
for i in range(len(nan_contained_files)):
    os.remove( 'csv_denoised/'+nan_contained_files[i][4:])
    